#В этом файле работы загрузим данные в модель наименьших квадратов, и посмотрим что и как у нас получится

In [27]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error
from scipy.stats import ttest_1samp
from scipy.stats import shapiro, levene, f_oneway, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
!pip install scikit-posthocs
import scikit_posthocs as sp


In [28]:
df=pd.read_excel("done_data.xlsx")
df.columns

Index(['Unnamed: 0', 'Фамилия Имя', 'Возраст игрока', 'Стоимость', 'Голы',
       'Ассисты', 'Голы с игры', 'Желтые карточки', 'красные карточки',
       'заработанные пенальти',
       ...
       'Клуб_Фламенго', 'Клуб_Фулхэм', 'Клуб_Челси', 'Клуб_Штуттгарт',
       'Клуб_Эвертон', 'Клуб_Ювентус', 'Национальность_Скандинав',
       'Национальность_Северная_Америка', 'Южная_Америка', 'Африканец'],
      dtype='object', length=127)

## Непосредственно модель

In [29]:
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
df = df.drop(columns=["Unnamed: 0", "Фамилия Имя"])
X = df.drop(columns="Стоимость")
for col in X.select_dtypes(include=['object']).columns:
    X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)
X = sm.add_constant(X)
y = df["Стоимость"]
X = X.applymap(lambda x: int(x) if isinstance(x, bool) else x)
model = sm.OLS(y, X)
result = model.fit()
mae = mean_absolute_error(y, result.predict(X))
rmse = np.sqrt(mean_squared_error(y, result.predict(X)))
latex_summary = result.summary().as_latex()
with open("kek", 'w') as f:
    f.write(latex_summary)
result.summary(), mae, rmse, summary_path



<ipython-input-29-a77472e55c2e>:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X = X.applymap(lambda x: int(x) if isinstance(x, bool) else x)


(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:              Стоимость   R-squared:                       0.769
 Model:                            OLS   Adj. R-squared:                  0.615
 Method:                 Least Squares   F-statistic:                     5.011
 Date:                Thu, 12 Dec 2024   Prob (F-statistic):           3.45e-22
 Time:                        23:11:14   Log-Likelihood:                -1168.7
 No. Observations:                 297   AIC:                             2575.
 Df Residuals:                     178   BIC:                             3015.
 Df Model:                         118                                         
 Covariance Type:            nonrobust                                         
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
 ----------

# Проверим гипотезы

### Первая гипотеза о переоценнености игроков АПЛ

In [30]:
X_epl = X[df["Лига_АПЛ"] == 1]
y_epl = y[df["Лига_АПЛ"] == 1]
predicted_value = result.predict(X_epl)
actual_value_epl = df["Стоимость"][df["Лига_АПЛ"] == 1]
overvaluation = actual_value_epl - predicted_value

print("Среднее overvaluation:", np.mean(overvaluation))

t_stat, p_value = ttest_1samp(overvaluation, 0)

print("Результаты t-теста по игрокам АПЛ:")
print("t-статистика:", t_stat)
print("p-значение:", p_value)

alpha = 0.05
if p_value < alpha:
    if np.mean(overvaluation) > 0:
        print("Отвергаем H0: игроки АПЛ переоценены.")
    else:
        print("Отвергаем H0: игроки АПЛ недооценены (фактическая стоимость меньше предсказанной).")
else:
    print("Нет статистически значимых доказательств переоценки или недооценки игроков АПЛ.")

Среднее overvaluation: 4.081079668023613e-12
Результаты t-теста по игрокам АПЛ:
t-статистика: 3.571638661762994e-12
p-значение: 0.9999999999971556
Нет статистически значимых доказательств переоценки или недооценки игроков АПЛ.


Первая гипотеза  не отвергается и мы не можем сказать, что игроки АПЛ переоценены

### Проверим вторую гипотезу о том , что атакующие более ценный товар, чем защитники и полузатники

In [31]:
position_columns = [col for col in df.columns if col.startswith('Позиция на поле_')]
attack_positions = [
    'Позиция на поле_Левый Вингер',
    'Позиция на поле_Правый Вингер',
    'Позиция на поле_Оттянутый нап.',
    'Позиция на поле_Центральный нап.'
]

midfield_positions = [
    'Позиция на поле_Опорный полузащитник',
    'Позиция на поле_Правый полузащитник',
    'Позиция на поле_Центр. полузащитник'
]

defence_positions = [
    'Позиция на поле_Левый защитник',
    'Позиция на поле_Правый защитник',
    'Позиция на поле_Центр. защитник'
]

def get_player_position(row):
    for c in position_columns:
        if row[c] == 1:
            return c
    return 'Unknown'
def categorize_position(pos):
    if pos in attack_positions:
        return 'Attack'
    elif pos in midfield_positions:
        return 'Midfield'
    elif pos in defence_positions:
        return 'Defence'
    else:
        return 'Other'
df['position'] = df.apply(get_player_position, axis=1)
df['position_group'] = df['position'].apply(categorize_position)

attack_values = df[df['position_group'] == 'Attack']['Стоимость'].dropna()
defence_values = df[df['position_group'] == 'Defence']['Стоимость'].dropna()
midfield_values = df[df['position_group'] == 'Midfield']['Стоимость'].dropna()
p_kruskal = kruskal(attack_values, defence_values, midfield_values)[1]
print("Kruskal-Wallis p-value:", p_kruskal)
if p_kruskal < 0.05:
    print("Есть статистически значимые различия между группами позиций.")
else:
    print("Нет статистически значимых различий между группами позиций.")
data = pd.concat([attack_values, defence_values, midfield_values], axis=0, keys=['Attack', 'Defence', 'Midfield'])
df_for_posthoc = pd.DataFrame({'values': data.values, 'groups': data.index.get_level_values(0)})
posthoc = sp.posthoc_dunn(df_for_posthoc, val_col='values', group_col='groups', p_adjust='bonferroni')
mean_att=df[df['position_group'] == 'Attack']['Стоимость'].mean()
mean_def=df[df['position_group'] == 'Defence']['Стоимость'].mean()
mean_mid=df[df['position_group'] == 'Midfield']['Стоимость'].mean()
print(posthoc,mean_att,mean_mid,mean_def)


Kruskal-Wallis p-value: 0.034758281225469756
Есть статистически значимые различия между группами позиций.
            Attack   Defence  Midfield
Attack    1.000000  0.034012  0.323787
Defence   0.034012  1.000000  1.000000
Midfield  0.323787  1.000000  1.000000 47.980582524271846 42.114285714285714 36.924731182795696
